In [4]:
import openai
import os

from dotenv import find_dotenv, load_dotenv

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, MessagesState, StateGraph, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

In [5]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
# chatbot instruction
MODEL_SYSTEM_MESSAGE = """
    You are a helpful assistant with memory that provides information about the user. 
    If you have memory for this user, use it to personalize your responses.
    Here is the memory (it may be empty): {memory}
"""

# create new memory from the chat history and any existing memory
CREATE_MEMORY_INSTRUCTION = """"
    You are collecting information about the user to personalize your responses.

    CURRENT USER INFORMATION:
    {memory}

    INSTRUCTIONS:
    1. Review the chat history below carefully
    2. Identify new information about the user, such as:
        - Personal details (name, location)
        - Preferences (likes, dislikes)
        - Interests and hobbies
        - Past experiences
        - Goals or future plans
    3. Merge any new information with existing memory
    4. Format the memory as a clear, bulleted list
    5. If new information conflicts with existing memory, keep the most recent version

    Remember: Only include factual information directly stated by the user. Do not make assumptions or inferences.

    Based on the chat history below, please update the user information:
"""

def call_model(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Load memory from the store and use it to personalize the chatbot's response.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("memory", user_id)
    key = "user_memory"
    existing_memory = store.get(namespace, key)

    if existing_memory:
        existing_memory_content = existing_memory.value.get('memory')        # value is a dictionary with a memory key
    else:
        existing_memory_content = "No existing memory found."

    # format the memory in the system prompt
    system_msg = MODEL_SYSTEM_MESSAGE.format(memory=existing_memory_content)
    
    # respond using memory as well as the chat history
    response = llm.invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": response}

def write_memory(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Reflect on the chat history and save a memory to the store.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")
        
    if existing_memory:
        existing_memory_content = existing_memory.value.get('memory')
    else:
        existing_memory_content = "No existing memory found."

    system_msg = CREATE_MEMORY_INSTRUCTION.format(memory=existing_memory_content)
    new_memory = llm.invoke([SystemMessage(content=system_msg)]+state['messages'])

    # overwrite the existing memory in the store 
    key = "user_memory"

    # write value as a dictionary with a memory key
    store.put(namespace, key, {"memory": new_memory.content})

In [7]:

graph = StateGraph(MessagesState)
graph.add_node("call_model", call_model)
graph.add_node("write_memory", write_memory)
graph.add_edge(START, "call_model")
graph.add_edge("call_model", "write_memory")
graph.add_edge("write_memory", END)

# store for across-thread memory
across_thread_memory = InMemoryStore()

# checkpointer for within-thread memory
within_thread_memory = MemorySaver()

# compile the graph with the checkpointer fir and store
graph = graph.compile(checkpointer=within_thread_memory, store=across_thread_memory)

In [8]:
# when we interact with the chatbot, we supply two things:
# within-thread memory: a thread ID for persisting the chat history.
# cross-thread memory: a user ID to namespace long-term memories to the user.
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

input_messages = [HumanMessage(content="Hi, my name is Yoona")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, my name is Yoona
================================== Ai Message ==================================

Hello Yoona! It's nice to meet you. How can I assist you today?


In [9]:
input_messages = [HumanMessage(content="I love super junior so much")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I love super junior so much
================================== Ai Message ==================================

That's great to hear! Super Junior is a popular K-pop group with a dedicated fan base. What do you love most about them?


In [10]:
# we using the MemorySaver checkpointer for within-thread memory. 
# this saves the chat history to the thread. 
# we can look at the chat history saved to the thread.
thread = {"configurable": {"thread_id": "1"}}
state = graph.get_state(thread).values
for m in state["messages"]: 
    m.pretty_print()

================================ Human Message =================================

Hi, my name is Yoona
================================== Ai Message ==================================

Hello Yoona! It's nice to meet you. How can I assist you today?
================================ Human Message =================================

I love super junior so much
================================== Ai Message ==================================

That's great to hear! Super Junior is a popular K-pop group with a dedicated fan base. What do you love most about them?


In [11]:
user_id = "1"
namespace = ("memory", user_id)
existing_memory = across_thread_memory.get(namespace, "user_memory")
existing_memory.dict()

{'value': {'memory': '**USER INFORMATION:**\n- Name: Yoona\n- Interest: Loves Super Junior'},
 'key': 'user_memory',
 'namespace': ['memory', '1'],
 'created_at': '2025-01-01T08:26:54.239352+00:00',
 'updated_at': '2025-01-01T08:26:54.239357+00:00'}

In [12]:
# we should see that the chatbot remembered the user's profile and used it to personalize the response.
# we supply a user ID for across-thread memory as well as a new thread ID
config = {"configurable": {"thread_id": "2", "user_id": "1"}}

input_messages = [HumanMessage(content="Hi! who do i like?")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

Hi! who do i like?
================================== Ai Message ==================================

Hi Yoona! You love Super Junior! If you have any specific questions or need more information about Super Junior, feel free to ask!
